In [2]:
import pandas as pd

In [3]:
def process_inactive_rows(df):
    '''
        폐업 및 삭제된 시설에 관한 데이터 제거
    '''
    _df = df.copy()
    if 'FCLTY_STATE_VALUE' in _df.columns:
        _df = _df[_df['FCLTY_STATE_VALUE'] != '폐업']
    if 'DEL_AT' in _df.columns:
        _df = _df[_df['DEL_AT'] != 'Y']
    return _df

def process_unnecessary_cols(df):
    '''
     불필요한 컬럼을 제거함
    '''
    _df = df.copy()
    drop_cols = [
        'FCLTY_MANAGE_LI_CD',
        'FCLTY_MANAGE_LI_NM',
        'ERDSGN_AT',
        'ATNM_CHCK_TRGET_AT',
        'DATA_ORIGIN_FLAG_CD',
        'DEL_AT',
    ]
    drop_cols = [i for i in drop_cols if i in _df.columns]
    _df.drop(columns=drop_cols, inplace=True)
    return _df


def process_duplicate_rows(df):
    '''
        시설명, 소재지, 시설유형명이 같은 데이터는 동일 시설로 간주하고 중복 제거
    '''
    _df = df.copy()
    def first_non_null(series):
        return series.dropna().iloc[0] if not series.dropna().empty else None
    _df = _df.groupby(
        ["FCLTY_NM", "POSESN_MBY_CTPRVN_CD", "INDUTY_NM"], as_index=False
    ).agg(first_non_null)
    return _df


In [4]:
public_facility_status = pd.read_csv('./data/raw/전국공공체육시설 데이터.csv',
                                     low_memory=False)
public_facility_status = process_inactive_rows(public_facility_status)
public_facility_status = process_unnecessary_cols(public_facility_status)
public_facility_status = process_duplicate_rows(public_facility_status)
public_facility_status.to_csv('./data/raw/전국공공체육시설 데이터_cleaned.csv', index=False)
public_facility_status.shape


(32112, 32)

In [5]:
all_facility_status = pd.read_csv('./data/raw/전국체육시설현황 데이터.csv',
                              on_bad_lines='skip',
                              low_memory=False)
# '폐업' 상태인 행 드롭
all_facility_status = process_inactive_rows(all_facility_status)
all_facility_status = process_unnecessary_cols(all_facility_status)
all_facility_status = process_duplicate_rows(all_facility_status)
all_facility_status.to_csv('./data/raw/전국체육시설현황 데이터_cleaned.csv', index=False)
all_facility_status.shape

(52716, 52)

In [6]:
all_facility_status['FCLTY_NM'].nunique()

49624

In [7]:
# 최종적으로 두 데이터프레임을 병합하고 결과값을 csv로 저장

merged_df = pd.merge(
    all_facility_status,
    public_facility_status,
    on=['FCLTY_NM', 'POSESN_MBY_CTPRVN_CD', 'INDUTY_NM'],
    how='outer',
    suffixes=('', '_public')
)

# 중복 컬럼 처리 - all_facility_status의 데이터 우선 사용
for col in merged_df.columns:
    if col.endswith('_public'):
        original_col = col[:-7]  # '_public' 제거
        # null 값인 경우에만 public_facility의 값으로 대체
        merged_df[original_col] = merged_df[original_col].fillna(merged_df[col])
        merged_df.drop(columns=[col], inplace=True)

merged_df.to_csv('./data/merged_facility_status.csv', index=False)
merged_df.shape


(53570, 63)

통합된 기존의 시설정보와 바우처 시설을 동일한 컬럼에 담는다.

In [6]:
merged_df = pd.read_csv('./data/merged_facility_status.csv', low_memory=False)

In [12]:
voucher_fac_df = pd.read_csv('./data/voucher_facilities.csv')
voucher_fac_df['voucher'] = True
voucher_fac_df.to_csv('./data/voucher_facilities.csv', index=False)

voucher_class_df = pd.read_csv('./data/voucher_classes.csv')
voucher_class_df["voucher"] = True
voucher_class_df.to_csv('./data/voucher_classes.csv', index=False)

In [ ]:
for col in voucher_fac_df.columns:
    if col not in merged_df.columns:
        print(col)

main_event_cd
city_nm
main_event_nm
city_cd
pres_nm
faci_daddr
row_num
facil_nm
facil_sn
road_addr
local_nm
faci_zip
local_cd
brno
facil_gbn_nm
res_telno
voucher


In [13]:
# 기존 퍼실리티 데이터에 맞게 컬럼명을 변경
voucher_fac_df["FCLTY_NM"] = voucher_fac_df["facil_nm"]
voucher_fac_df["FCLTY_TY_NM"] = voucher_fac_df["main_event_nm"]
voucher_fac_df["CTPRVN_NM"] = voucher_fac_df["city_nm"]
voucher_fac_df["CTPRVN_CD"] = voucher_fac_df["city_cd"]
voucher_fac_df["FULL_ADDR"] = voucher_fac_df["road_addr"] + voucher_fac_df["faci_daddr"]
voucher_fac_df["RSPNSBLTY_TEL_NO"] = voucher_fac_df["res_telno"]

In [14]:
# 변경한 컬럼과 불필요한 컬럼을 드롭해서, 병합을 준비
columns_to_drop = [
    "facil_nm",
    "main_event_nm",
    "city_nm",
    "city_cd",
    "road_addr",
    "faci_daddr",
    "res_telno",
    "local_nm",
    "faci_zip",
    "local_cd",
    "facil_gbn_nm",
    "row_num",
    'main_event_cd',
]

filtered_drop_cols = [col for col in columns_to_drop if col in final_facility_df.columns]
voucher_fac_df.drop(columns=filtered_drop_cols, inplace=True)

In [16]:
voucher_fac_df.to_csv('./data/voucher_facilities_cols_processed.csv', index=False)

In [18]:
for name in voucher_fac_df['FCLTY_NM'].unique():
    if name in merged_df['FCLTY_NM'].unique():
        print(name)

BK승마랜드
건국대창헌태권도
구슬당구장
21세기체대태권스쿨
JY태권도
Jun태권스쿨
J다이어트 복싱 체육관
KAFA휘트니스
KG태권짐
KTC 김태균 태권도장
KTI월드태권스쿨
KTI한울태권도
제이제이
LTA 태권도장
경희대라이온태권도
NTS태권도(창동1호점)
ONE도장
SAEM태권도장
SB휘트니스
MAX태권도장
MBA태권도장
MNT 태권도 스포츠센터
MTA 미래태권도장
MTA 태권도
MTA 태권도장
MTA태권도
MTA태권도장
MTA화랑태권도장
MVP당구장
CHAN 복싱클럽
ECN태권도장
FITNESS이루다
GB휘트니스
GMS정훈도장
H3복싱체육관
HHIM 힘 복싱스쿨
H휘트니스
I CAN 늘품 태권도장
I CAN 대성태권도
J&J휘트니스
J2M SPORTS
JM휘트니스
JS Boxing Gym
JSK경호태권도아카데미
THE 힘찬태권도장
TIA태권도
TOP태권도
The조은태권도장
U can 태권도
VIP주니어센터
VIP태권도
푸른휘트니스
가흥태권도장
갈산용인대태권도체육관
감계중앙태권도
가족태권도체육관
감사랑 리더십 태권도장
XPL 태권도
e고려힐링태권도
가경검도관
가동태권도
가람검도관
가람체육관
가람태권도
가야검도관
가양태권도장
가온 태권도장
가온누리
가온누리 명성태권도장
가온누리 용인대 태권도장
가람태권도장
가야중앙체육관
가온누리태권도장
가온체형교정운동센터
가온태권도
가온태권도장
가운태권도
동양태권도스쿨
태권마루
SJ태권도예절관
SM골프아카데미
SM댄스아카데미
가재울검도장
SY피트니스짐
강촌키즈태권도
강한 태권도
강한대희태권도장
강한아이 태권도장
강한아이 한진체육관
해동검도
강한태권도
강한태권도장
강해라 태권도장
강호유도관
강호체육관
강호태권도
강호태권도장
계명태권스쿨
강화검도관
거산MMA태권도장
거산체육관
거송유도관
거성태권도
경무고려태권도장
경무체육관
경무태권도체육관
경무키즈태권도효도관
경북대 승리 태권도
경북대튼튼태권도
경성태권도
경연태권도
경우체육관
경원체육관
경일태권도
거인태권도
거창검도관
건강한태권도
건강해GYM
건강한태권도교육관
건국태권도
건무태권

KeyboardInterrupt: 